In [1]:
import pandas as pd
from scipy.stats import pearsonr
import numpy as np

Here in the 1st function. The primary assumption is that dataset must be having date column/columns in it because we are going to work on dates only so this is the requirement.
Secondally in my dataset i'm having Date1 Date2 and Date3 in my dataset, this is not a fixed thing for every dataset so this thing must be changed/modified based on the features names we can have(e.g sometime we can have date/DATE,Years and others). 


So basically previously i have three date columns in my dataset namely Date1 Date2 and Date3, after the execution it is now converted into six columns such as three previous columns(Date1 Date2 and Date3) and three new columns(new1  new2  new3)
new1=date1-date2,
new2=date2-date3,
new3=date3-date1.

In [26]:
import pandas as pd
def my_date_func():
    #path=r"C:\Users\shalo\Desktop\ML stuffs\test\lannet\dates.csv"#path of dataset
    df=pd.read_csv("/dates.csv")#1.data input as dataFrame
    df_old=df.copy()#copying main data for refrence
    dates=[feature for feature in df.columns if 'Date' in feature ]#2.identifying which columns having date(*in my case ,I'm having "Date" in my dataset date columns names,this can be diffrent of diffrent datasets)
    #above written loop will traverse through each columns of the dataframe and will pick only Columns having "Date" in them
    format = "%d-%m-%Y"#Validiating columns(now we will set a particular format of date for validiating the dates features, this will block invalid entries)
    for i in range(len(dates)):#the loop will run as per the number of date features we got from the dataset
        for j in range(len(df[dates[i]])):#this loop will traverse through each and every entries of each and every date columns to check them
            date_string=df[dates[i]].values[j]#will pick up each entry
            try:
                datetime.datetime.strptime(date_string, format)#if entries found to be in the specified format then we will keep that entry
            except:
                df[dates[i]].drop([j])#if the entry found out of the specified format(invalid entry) then that entry be removed
    df[dates].dropna(inplace=True)#after dropping invalid entries we will be having lots of nan values in our data set so those nan values must be removed 
    for k in range(len(dates)):
        df[dates[k]]=pd.to_datetime(df[dates[k]])#Making same format for diffrent formats and preparing them for diffrence
    #4.now diffrence
    for l in range(len(dates)):
        try:
            df['new'+str(l+1)]=(df[dates[l]]-df[dates[l+1]]).dt.days#date1-date2,date2-date3
        except:
            df['new'+str(l+1)]=(df[dates[l]]-df[dates[l-2]]).dt.days#date3-date1
    return df,df_old

In [27]:
new,old=my_date_func()

In [28]:
print("*************New dataframe************")
new.head()

*************New dataframe************


,Date1,Date2,Date3,new1,new2,new3
0,2016-11-21,2018-07-13,2020-02-02,-599,-569,1168
1,2016-11-22,2018-07-14,2020-03-02,-599,-597,1196
2,2016-11-23,2018-07-15,2020-04-02,-599,-627,1226
3,2016-11-24,2018-07-16,2020-05-02,-599,-656,1255
4,2016-11-25,2018-07-17,2020-06-02,-599,-686,1285


In [29]:
print("*************Old dataframe************")
old.head()

*************Old dataframe************


,Date1,Date2,Date3
0,21-11-2016,13-07-2018,02-02-2020
1,22-11-2016,14-07-2018,03-02-2020
2,23-11-2016,15-07-2018,04-02-2020
3,24-11-2016,16-07-2018,05-02-2020
4,25-11-2016,17-07-2018,06-02-2020


In 2nd function. In this we are seprating target column first and then we will check pearson correlation of all other columns with this seprated target feature. If correlation of target column and any other feature is less than 0.5 that column will be dropped.
One thing to remember name of target column for each data set is diffrent, as in my case name of target feature is "SalePrice" so this is varying(can be changed as per the nature of dataset).
Also to prevent dropping of most columns, i am using 70% of total columns will be preserved and only 30% columns will only be dropped for example if we have 10 columns in our data set(as i'm using in dummy dataset)
and all 10 columns are having correlation less than 0.5 so only 3 columns would be dropped rest 7 columns would be preserved.

In [23]:
import pandas as pd
from scipy.stats import pearsonr
def drop_cols():
    path1=r"C:\Users\shalo\Desktop\ML stuffs\machine_learning_pipelines\Advanced-House-Price-Prediction--master\train.csv" #path of csv file/dataset
    data=pd.read_csv(path1)# Taking dataframe as input
    TARGET_CLASS=data["SalePrice"]# Seprating the dependent/target feature from the data set(this must be changed for every data set beacuse every dataset can have a diffrent name of target features)
    dep_cols=data.drop("SalePrice", axis=1)#taking all the dependent features together
    cols=[feature for feature in dep_cols.columns if dep_cols[feature].dtypes!='O']#in dependents features we can have categorical features also so we will be taking only numerical features to calculate correlation with target feature(here dtypes!='O' means data type is not object type)
    #above written loop will traverse through each and every dependent column and will take only numerical columns
    data1=data[cols]#data1 dataframe will have only numerical dependent features
    c=0#nothing but a counter to count how many columns have Pearson correlation more than 0.85 out of all the numerical columns
    li=[]#will store the value of Pearson correlation
    for i in range(len(cols)):#will traverse through each column having numerical datatype
        median_value=data[cols[i]].median()
        data[cols[i]].fillna(median_value,inplace=True)#if any column is having null values we will fill that with taking median of that column only
        corr, _ = pearsonr(data[cols[i]],TARGET_CLASS)#now the main task is to take pearson correlation of target with any other numerical column
        if corr<0.5:#if correlation of that particular column and target column comes less than 0.5 that column will be removed from data1 dataframe which is having only numerical features(we are not dropping from main dataset just to visualise before and after effect )
            if (len(data1.columns)/len(cols))>0.7:#this will make sure only least amount of varibales get dropped(here i'm taking only 70% of the total columns)
                data1=data1.drop(cols[i],axis=1)
                c+=1#will count
                li.append(corr)
            else:
                c+=1
                li.append(corr)
            
        else:#other wise we will keep that column if Pearson correlation more than  0.5
            continue
    print("Total number of previous columns:",len(cols))#it will tell us all the numerical columns previosly we had
    print("Total number of new columns:", len(data1.columns))#since we are dropping columns from data1 dataframe which is having only numerical features so will count the all those columns which are now left
    if len(cols)==len(data1.columns):
        print("no column have Pearson correlation less than 0.5,So no column is dropped")
    else:
        print("Total",len(cols)-len(data1.columns),"columns dropped")
        print("Total number of columns having Pearson correlation less than 0.5 are/is:",c)
        print("Pearson correlation as follows:", li)
    data1["Target"]=data["SalePrice"]#since data1 dataframe having only numerical dependent features so we will target column also into it
    return data, data1

In [24]:
old,new=drop_cols()

Total number of previous columns: 37
Total number of new columns: 25
Total 12 columns dropped
Total number of columns having Pearson correlation less than 0.5 are/is: 27
Pearson correlation as follows: [-0.021916719443431112, -0.0842841351265952, 0.33477085313976007, 0.2638433538714057, -0.07785589404867801, 0.47261449900457375, 0.38641980624215333, -0.011378121450215141, 0.21447910554696892, 0.3193338028320678, -0.02560613000067956, 0.22712223313149424, -0.016844154297359012, 0.28410767559478256, 0.16821315430073996, -0.1359073708421412, 0.46692883675152774, 0.46675365236333954, 0.3244134445681299, 0.3158562271160553, -0.12857795792595675, 0.04458366533574841, 0.1114465711429112, 0.0924035494918732, -0.02118957964030326, 0.04643224522381934, -0.028922585168730326]


In [25]:
print("*************Old dataframe************")
old.head()

*************Old dataframe************


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [26]:
print("*************New dataframe************")
new.head()

*************New dataframe************


,OverallQual,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,GrLivArea,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,Target
0,7,2003,2003,856,856,1710,0,2,1,3,...,0,61,0,0,0,0,0,2,2008,208500
1,6,1976,1976,1262,1262,1262,1,2,0,3,...,298,0,0,0,0,0,0,5,2007,181500
2,7,2001,2002,920,920,1786,0,2,1,3,...,0,42,0,0,0,0,0,9,2008,223500
3,7,1915,1970,756,961,1717,0,1,0,3,...,0,35,272,0,0,0,0,2,2006,140000
4,8,2000,2000,1145,1145,2198,0,2,1,4,...,192,84,0,0,0,0,0,12,2008,250000
